In [1]:
import sys
print(sys.version)
from difflib import SequenceMatcher
import numpy as np

3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)]


In [17]:
import pandas as pd
master_csv = pd.read_csv(filepath_or_buffer="jidosyo_gakusan_20230508_master.csv", encoding="utf8", sep=",")

In [18]:
master_csv.head(10)

,JAN,SHOSINM,KANSU,ITMCD,DAIGENRECD,CHUGENRECD,SHOGENRECD,SIKIRIBANCD,ITMNM,DAIGENRENM,...,CHOSHANM,SHPNSHACD,SHPNSHANM,SAKUHINCD,SAKUHINNM,SOSHONM,HATUBAIYMD,ANNUAL_SALES,SUBLABEL,SERIESNM
0,9784046322050,サキヨミ！ トラブルまみれのデート作戦！,８,1076,16,55,1,15,書籍,児童書,...,七海まち,4,ＫＡＤＯＫＡＷＡ,43182126,サキヨミ！,角川つばさ文庫,20230208,2659,NaN,NaN
1,9784410107276,チャート式解法と演習数学２＋Ｂ 新課程,NaN,1076,19,31,11,11,書籍,学習参考書,...,チャート研究所,410,数研出版,43371515,チャート式解法と演習数学２＋Ｂ 新課程,NaN,20221025,1656,NaN,NaN
2,9784892880865,北辰のかこもん 数学 ２０２２年度中３北辰テスト過去問題集,NaN,1076,19,21,5,11,書籍,学習参考書,...,ｍｉｏ,89288,北辰図書出版,43412911,北辰のかこもん 数学 ２０２２年度中３北辰テスト過去問題集,NaN,20230322,2269,NaN,NaN
3,9784046321831,ぜったいバレちゃいけません！！！ ハッピーエンドはゆずれない！,６,1076,16,55,1,15,書籍,児童書,...,水無仙丸,4,ＫＡＤＯＫＡＷＡ,43210129,ぜったいバレちゃいけません！！！,角川つばさ文庫,20221013,1534,NaN,NaN
4,9784065301821,氷の上のプリンセス シニア編,３,1076,16,55,1,11,書籍,児童書,...,風野潮,6,講談社,43308401,氷の上のプリンセス シニア編,講談社青い鳥文庫,20221214,1433,NaN,NaN
5,9784046321879,スイッチ！ 大キライなはずなのに！？,１１,1076,16,55,1,15,書籍,児童書,...,深海ゆずは,4,ＫＡＤＯＫＡＷＡ,42943339,スイッチ！,角川つばさ文庫,20221109,3376,NaN,NaN
6,9784813780755,イケメン王子×４は、地味子ちゃんを溺愛したい。 ヒミツの甘々カップル誕生編,NaN,1076,16,55,1,91,書籍,児童書,...,ゆいっと,8137,スターツ出版,43379328,イケメン王子×４は、地味子ちゃんを溺愛したい。 ヒミツの甘々カップル誕生編,野いちごジュニア文庫,20221220,1179,NaN,NaN
7,9784791631612,頭がよくなる！ はじめての寝るまえ１分おんどく,NaN,1076,16,11,2,1,書籍,児童書,...,加藤俊徳,7916,西東社,43408299,頭がよくなる！ はじめての寝るまえ１分おんどく,NaN,20230405,2602,NaN,NaN
8,9784813780854,ウタイテ！ 目指せ、ファンイベント開催！ ヒミツの特訓は溺愛のはじまり,４,1076,16,55,1,91,書籍,児童書,...,＊あいら＊,8137,スターツ出版,43313006,ウタイテ！,野いちごジュニア文庫,20230320,6136,NaN,NaN
9,9784046322203,サバイバー！！ アテンション、敵のアジトから生還せよ！,５,1076,16,55,1,15,書籍,児童書,...,あさばみゆき,4,ＫＡＤＯＫＡＷＡ,43261596,サバイバー！！,角川つばさ文庫,20230412,1626,NaN,NaN


In [19]:
df = pd.read_csv(filepath_or_buffer="child_series_moto__shpnshanm_seriesnm.csv", encoding="utf8", sep=",")
print(df)

     SHPNSHANM       SERIESNM
0       あいうえお館    ウルトラかいじゅう絵本
1       あいうえお館         ウルトラマン
2        あかね書房      おばけのポーちゃん
3        あかね書房  いぬうえくんとくまざわくん
4        あかね書房          日本の神話
...        ...            ...
3560     Ｚ会編集部     グレードアップワーク
3561     Ｚ会編集部     Ｚ会中学受験シリーズ
3562     Ｚ会編集部     グレードアップ問題集
3563     Ｚ会編集部     Ｚ会小学英語シリーズ
3564     Ｚ会編集部     グレードアップドリル

[3565 rows x 2 columns]


In [20]:
seriesnm = {}
shpnshanm = ''
shpnshanm = set(df['SHPNSHANM'].values.tolist())
for l in shpnshanm:
    seriesnm[l] = []

for index, row in df.iterrows():
    seriesnm[row['SHPNSHANM']].append(row['SERIESNM'])

In [11]:
print(seriesnm['あいうえお館'])

['ウルトラかいじゅう絵本', 'ウルトラマン']


In [21]:
# input_nameに書籍名、shpnshanmに出版社名をわたす
# 渡した書籍名に、seriesnmの出版社に紐づくシリーズ名があればそれを返す。
# 例　
# input_name = サキヨミ！　恋と決意の文化祭
# shpnshanm = ＫＡＤＯＫＡＷＡ
# の場合、seriesnm['ＫＡＤＯＫＡＷＡ']にはＫＡＤＯＫＡＷＡに紐づくシリーズ名がすべて含まれている
# そのうち「サキヨミ！」はinput_nameの「サキヨミ！　恋と決意の文化祭」にシリーズ名が含まれているため
# check_inputname_in_seriesnm(input_name, shpnshanm)を実行すると「サキヨミ！」が戻り値となる

def check_inputname_in_seriesnm(input_name, shpnshanm):

    if not shpnshanm in seriesnm:
        return
    
    for sn in seriesnm[shpnshanm]:
        # if input_name in sn:
        if sn in input_name:
            return sn
        
    return ''

In [25]:
print('サキヨミ！' in 'サキヨミ！　恋と決意の文化祭')

True


In [13]:
print(check_inputname_in_seriesnm('サキヨミ！　恋と決意の文化祭', 'ＫＡＤＯＫＡＷＡ'))

サキヨミ！


In [15]:
print(seriesnm['ＫＡＤＯＫＡＷＡ'])
print('サキヨミ！' in seriesnm['ＫＡＤＯＫＡＷＡ'])

['ぼくと先輩のマジカル・ライフ', '角川まんが学習シリーズ', '人気大学過去問シリーズ', 'らくがき☆ポリス', 'ビブリア古書堂の事件手帖', '逆転裁判逆転', '空のふしぎがわかる', '怪盗レッドＴＨＥＦＩＲＳＴ', '三毛猫ホームズ', '悪夢ちゃん', 'トツゲキ！？地獄ちゃんねる', 'アナと雪の女王', '告白実行委員会', 'いみちぇん！', 'ピアノ・カルテット', 'ちいさなプリンセスソフィア', '怪盗探偵山猫', 'からくり探偵団', 'ディズニー_角川アニメ絵本', '恐怖コレクター', '幽霊探偵ハル', 'サッカーボーイズ', '少年探偵\u3000響', '0.01', 'バッテリー', 'ここは妖怪おたすけ委員会', '宮沢賢治童話集', 'アルセーヌ・ルパン', '超吉ガール', 'オンライン！', 'ドギーマギー動物学校', 'プレイボール', '怪盗レッド', '角川まんが超科学シリーズ', 'あけてびっくり', 'ネオ里見八犬伝', 'ハルチカ', 'モンスターハンター', '伝えたい、原爆のこと', '２年Ａ組探偵局', '５分で読書', 'ほっぺちゃん', 'シートン動物記', '絶体絶命ゲーム', 'サクラダリセット', '新訳・ナルニア国物語', 'パンどろぼう', 'どっちが強い！？シリーズ', '世界一クラブ', 'ぷよぷよ', '赤毛のアン', 'いっしょにくらそ。', '五年霊組こわいもの係', 'プリンセス・ストーリーズ', 'ジュニア空想科学読本', 'ソライロ♪プロジェクト', 'イケカジなぼくら', '魔女犬ボンボン', 'モンハン日記', '新訳・ドリトル先生', '恋する新選組', 'ぼくらの', 'コウペンちゃん', '星のカービィ', '角川まんが学習シリーズ\u3000まんが人物伝', '角川の集める図鑑ＧＥＴ！', 'ゲゲゲの鬼太郎おばけ塾', 'Ｓａｓｓｙのちいくえほん', 'Ｓａｓｓｙのあかちゃんえほん', 'たったひとりのサバイバル・ゲーム！', '角川アニメ絵本', 'ディズニー', 'サッカク探偵団', '暗号クラブ', 'マジック・ツリーハウス', '世にも奇妙な商品カタログ', 'スイッチ！', '少女マンガじゃない！', '本当はこわい話', 'ディズニーツムツム

In [22]:
# 完全に含まれているかを確認
# SHOSINM, SIKIRIBANNM, SOSHONM, SUBLABEL

complete_seriesnm_candidate_list = []
for index, row in master_csv.iterrows():
    complete_seriesnm_candidate_list.append(check_inputname_in_seriesnm(row['SHOSINM'], row['SHPNSHANM']))
    
master_csv['SERIESNM_shoshi'] = complete_seriesnm_candidate_list

In [23]:
# 完全に含まれているかを確認
# SHOSINM, SIKIRIBANNM, SOSHONM, SUBLABEL

complete_seriesnm_candidate_list = []
for index, row in master_csv.iterrows():
    complete_seriesnm_candidate_list.append(check_inputname_in_seriesnm(row['SIKIRIBANNM'], row['SHPNSHANM']))
    
master_csv['SERIESNM_sikiriban'] = complete_seriesnm_candidate_list

In [24]:
# 完全に含まれているかを確認
# SHOSINM, SIKIRIBANNM, SOSHONM, SUBLABEL

complete_seriesnm_candidate_list = []
for index, row in master_csv.iterrows():
    if pd.isnull(row['SOSHONM']):
        complete_seriesnm_candidate_list.append('')
    else:
        complete_seriesnm_candidate_list.append(check_inputname_in_seriesnm(row['SOSHONM'], row['SHPNSHANM']))
    
master_csv['SERIESNM_sosho'] = complete_seriesnm_candidate_list

In [25]:
# 完全に含まれているかを確認
# SHOSINM, SIKIRIBANNM, SOSHONM, SUBLABEL

complete_seriesnm_candidate_list = []
for index, row in master_csv.iterrows():
    if pd.isnull(row['SUBLABEL']):
        complete_seriesnm_candidate_list.append('')
    else:
        complete_seriesnm_candidate_list.append(check_inputname_in_seriesnm(row['SUBLABEL'], row['SHPNSHANM']))
    
master_csv['SERIESNM_sublabel'] = complete_seriesnm_candidate_list

In [26]:
#書籍名が既存のシリーズ名と部分一致するかを確認
# shpnshanmがseriesnmの出版社に含まれてなければ終了
# 含まれている場合、書籍名とシリーズ名が一番長く部分一致するシリーズ名を
# シリーズ候補名として返す

def extract_seriesnm_candidate(input_name, shpnshanm):
    seriesnm_candidate = ''
    max_match_size = 0
    
    if not shpnshanm in seriesnm:
        return
    
    for sn in seriesnm[shpnshanm]:
        match = SequenceMatcher(None, input_name, sn).find_longest_match(0, len(input_name), 0, len(sn))

        # 2文字以上一致するか確認
        if max_match_size < match.size and match.size >= 2:
            seriesnm_candidate = sn
            max_match_size = match.size
    
    return seriesnm_candidate

In [27]:
# 既存のシリーズ名との部分一意
# SHOSINM, SIKIRIBANNM, SOSHONM, SUBLABEL

seriesnm_candidate_list = []
for index, row in master_csv.iterrows():
    seriesnm_candidate_list.append(extract_seriesnm_candidate(row['SHOSINM'], row['SHPNSHANM']))

master_csv['seriesnm_candidate_shoshi'] = seriesnm_candidate_list

In [28]:
# 既存のシリーズ名との部分一意
# SHOSINM, SIKIRIBANNM, SOSHONM, SUBLABEL

seriesnm_candidate_list = []
for index, row in master_csv.iterrows():
    seriesnm_candidate_list.append(extract_seriesnm_candidate(row['SIKIRIBANNM'], row['SHPNSHANM']))

master_csv['seriesnm_candidate_sikiriban'] = seriesnm_candidate_list

In [29]:
# 既存のシリーズ名との部分一意
# SHOSINM, SIKIRIBANNM, SOSHONM, SUBLABEL

seriesnm_candidate_list = []
for index, row in master_csv.iterrows():
    if pd.isnull(row['SOSHONM']):
        seriesnm_candidate_list.append('')
    else:
        seriesnm_candidate_list.append(extract_seriesnm_candidate(row['SOSHONM'], row['SHPNSHANM']))

master_csv['seriesnm_candidate_sosho'] = seriesnm_candidate_list

In [30]:
# 既存のシリーズ名との部分一意
# SHOSINM, SIKIRIBANNM, SOSHONM, SUBLABEL

seriesnm_candidate_list = []
for index, row in master_csv.iterrows():
    if pd.isnull(row['SUBLABEL']):
        seriesnm_candidate_list.append('')
    else:
        seriesnm_candidate_list.append(extract_seriesnm_candidate(row['SUBLABEL'], row['SHPNSHANM']))

master_csv['seriesnm_candidate_sublabel'] = seriesnm_candidate_list

In [31]:
master_csv.to_csv('jidosyo_gakusan_output_20230508.csv')